<a href="https://colab.research.google.com/github/Manvit07/FRIDAY_FOSS/blob/main/Friday_code_fixer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.7 MB/s eta 0:00:00


In [2]:
!huggingface-cli logout

Not logged in!


In [3]:
!pip install pandas nltk transformers torch tqdm scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [4]:
!pip install transformers datasets torch pandas nltk tqdm scikit-learn
!pip install sentencepiece  # Required for CodeT5 tokenizer

In [5]:
!git clone https://github.com/salesforce/CodeT5.git
%cd CodeT5/CodeT5+

Cloning into 'CodeT5'...
remote: Enumerating objects: 295, done.
remote: Counting objects: 100% (140/140), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 295 (delta 105), reused 65 (delta 65), pack-reused 155 (from 1)
Receiving objects: 100% (295/295), 10.91 MiB | 7.86 MiB/s, done.
Resolving deltas: 100% (151/151), done.
/content/CodeT5/CodeT5+


In [6]:

!python run_gen.py \
    --model_name_or_path Salesforce/codet5p-base \
    --do_train \
    --do_eval \
    --train_file ../dataset.csv \
    --output_dir ./codet5p-fine-tuned \
    --per_device_train_batch_size 8 \
    --per_device_eval_batch_size 8 \
    --num_train_epochs 10 \
    --learning_rate 5e-5 \
    --save_strategy epoch \
    --evaluation_strategy epoch \
    --logging_steps 10 \
    --save_total_limit 2 \
    --fp16

python3: can't open file '/content/CodeT5/CodeT5+/run_gen.py': [Errno 2] No such file or directory


In [7]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from transformers import AutoTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from transformers import T5ForConditionalGeneration,T5Tokenizer,Trainer,TrainingArguments,AutoModelForSeq2SeqLM, AutoTokenizer
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
from datasets import load_dataset

import os


In [8]:
import json

In [10]:
df = pd.read_csv("/content/dataset.csv")

In [12]:
# ✅ Load the dataset from JSON
dataset = load_dataset("json", data_files="/content/dataset.json")

# ✅ Check the structure of the dataset
print(type(dataset))  # Should output <class 'datasets.DatasetDict'>

# ✅ Display dataset structure
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

<class 'datasets.dataset_dict.DatasetDict'>
DatasetDict({
    train: Dataset({
        features: ['_key', 'trigger_function', 'explanation', 'corrected_code'],
        num_rows: 37
    })
})


In [13]:
columns_to_keep = ["trigger_function", "explanation","corrected_code"]
df = df[columns_to_keep]

In [15]:
dataset = load_dataset("json", data_files="/content/dataset.json")

In [16]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
model_name = "Salesforce/codet5p-220m"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

In [23]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/37 [00:00<?, ? examples/s]

In [24]:
model = AutoModelForSeq2SeqLM.from_pretrained("Salesforce/codet5p-220m").to("cuda")

config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/446M [00:00<?, ?B/s]

In [27]:
def tokenize_function(examples):
    model_inputs = tokenizer(examples["trigger_function"], padding="max_length", truncation=True, max_length=128)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["corrected_code"], padding="max_length", truncation=True, max_length=128)

    model_inputs["labels"] = labels["input_ids"]  # ✅ Assign corrected code as decoder input
    return model_inputs


In [25]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# ✅ Load CodeT5+ Model
model = AutoModelForSeq2SeqLM.from_pretrained("Salesforce/codet5p-220m").to("cuda")

Map:   0%|          | 0/37 [00:00<?, ? examples/s]

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



In [30]:
training_args = TrainingArguments(
    output_dir="./codet5p-fine-tuned",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=16,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",
)

# ✅ Initialize Trainer (Fix KeyError)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],  # ✅ Correct way to access the dataset
    eval_dataset=tokenized_datasets["train"],   # ✅ Change this if you have a separate validation set
)


# ✅ Start Training
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,6.778563
2,7.422100,6.324395
3,7.422100,5.937444
4,6.116400,5.512852
5,6.116400,5.168508
6,5.434200,4.941439
7,5.434200,4.809301
8,5.048100,4.738576
9,5.048100,4.682240
10,4.878300,4.644464


TrainOutput(global_step=80, training_loss=5.387778949737549, metrics={'train_runtime': 415.4375, 'train_samples_per_second': 1.425, 'train_steps_per_second': 0.193, 'total_flos': 90125767802880.0, 'train_loss': 5.387778949737549, 'epoch': 16.0})

In [33]:
def fix_code(error_text):
    model.eval()  # Set model to evaluation mode
    inputs = tokenizer("Fix this code: " + error_text, return_tensors="pt", padding="max_length", truncation=True).to("cuda")

    with torch.no_grad():
        output_ids = model.generate(inputs["input_ids"], max_length=128, num_beams=5)

    fixed_code = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return fixed_code if fixed_code.strip() else "⚠️ No meaningful suggestion generated!"

# ✅ Test with an incorrect example
error_example = "for i in range(5 print(i)"
fix_suggestion = fix_code(error_example)
print(f"💡 Fix Suggestion: {fix_suggestion}")


💡 Fix Suggestion: 
Fix this code: for i in range(5) print(i)
